# Deploy a GPTQ Quantized model on SageMaker with Hugging Face TGI

---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook.

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-2/inference|generativeai|llm-workshop|deploy-gptq-quant-tgi|deploy_gptq_quant_tgi.ipynb)

---

In this notebook we will deploy a pre-quantized 13 billion parameter [Llama 2 Chat model](https://huggingface.co/TheBloke/Llama-2-13B-chat-GPTQ). 

The original models is stored and served in half-precision fp16 format which translates to 2 bytes per parameter. Given that the model has 13 billion parameters, the model size translates 26GB which is too large to fit in the memory of a single A10 GPU which has only 24GB of memory. This requires us to use a more expensive multi-gpu instance such as a `ml.g5.12xlarge`. An alternative is to quantize the model which can significantly reduce the amount of VRAM required to host the model.  

In this notebook, we will deploy a 13 billion parameter model that has been pre-quantized to 4-bits using [GPTQ algorithm](https://arxiv.org/abs/2210.17323). With 4bit quantization the amount of memory per parameter is reduced from 2 bytes to 4 bits (0.5 bytes) which translates to a 75% reduction in memory footprint. This allows us to host the model on a single A10 GPU instance, such as a `ml.g5.xlarge`, which is significantly cheaper than a multi-gpu instance. As a disclaimer, quantization does result in a slight drop in model accuracy. However, the drop in accuracy is small and the model is still able to generate coherent responses, but it is important to evaluate the model on your use case.  


*Llama 2 is licensed under the LLAMA 2 Community License, Copyright (c) Meta Platforms, Inc. All Rights Reserved

In [ ]:
%pip install -Uq sagemaker

In [ ]:
import boto3
import sagemaker
import json
from sagemaker import Model
from sagemaker.huggingface import get_huggingface_llm_image_uri
import time
from pathlib import Path

boto3_session = boto3.session.Session()

smr = boto3_session.client(
    "sagemaker-runtime"
)  # sagemaker runtime client for invoking the endpoint
sm = boto3_session.client("sagemaker")  # sagemaker client for creating the endpoint


role = sagemaker.get_execution_role()  # execution role for the endpoint

sess = sagemaker.session.Session(
    boto3_session, sagemaker_client=sm, sagemaker_runtime_client=smr
)  # sagemaker session for interacting with different AWS APIs

bucket = (
    sess.default_bucket()
)  # sagemaker session for interacting with different AWS APIs
region = sess._region_name  # region name of the current SageMaker Studio environment

We will be using the Hugging Face Text Generation Inference (TGI) Container which runs the optimized [TGI](https://github.com/huggingface/text-generation-inference) LLM hosting solution from HuggingFace


In [ ]:
# retrieve the llm image uri
llm_image_uri = get_huggingface_llm_image_uri("huggingface", version="1.0.3", session=sess)

The helper function below will deploy the model to a SageMaker endpoint.

In [ ]:
def deploy_model(
    endpoint_name,
    instance_type,
    env=None,
    image_uri=None,
    model_artifact=None,
    s3_bucket=None,
    s3_prefix=None,
    wait=True,
):
    """Uploads the model artifact to S3 and deploys the model to SageMaker."""
    if model_artifact:
        code_artifact = sess.upload_data(model_artifact, s3_bucket, s3_prefix)
        print(f"Inference Code tar ball uploaded to --- > {code_artifact}")
    else:
        code_artifact = None

    model = Model(
        sagemaker_session=sess,
        image_uri=image_uri,
        model_data=code_artifact,
        env=env,
        role=role,
    )

    model.deploy(
        initial_instance_count=1,
        instance_type=instance_type,
        endpoint_name=endpoint_name,
        wait=wait,
    )

    return model

The repo maintained by [TheBloke](https://huggingface.co/TheBloke) contains many pre-quantized models. You can also easily quantize your own models with GPTQ by using the [AutoGPTQ](https://github.com/PanQiWei/AutoGPTQ) library.

In [ ]:
# create unique names for the endpoints
llm_endpoint_name = sagemaker.utils.name_from_base("llama2")

llm_env = {
    "HF_MODEL_ID": "TheBloke/Llama-2-13B-chat-GPTQ",  # model_id from hf.co/models
    "SM_NUM_GPUS": "1",  # Number of GPU used per replica
    "REVISION": "gptq-4bit-32g-actorder_True",  # model revision (main is fastest while gptq-4bit-32g-actorder_True provides best quality at 4-bits)
    "MAX_INPUT_LENGTH": "3072",  # Max length of input text
    "MAX_TOTAL_TOKENS": "4096",  # Max length of the generation (including input text)
    "MAX_BATCH_TOTAL_TOKENS": "8192",  # Limits the number of tokens that can be processed in parallel during the generation
    "HF_MODEL_QUANTIZE": "gptq",  # serve a pre-quantized model,
    "GPTQ_BITS": "4",
    "GPTQ_GROUPSIZE": "32",
    "DISABLE_EXLLAMA": "True", #Set to False for a 30% performance gain when this is fixed https://github.com/huggingface/text-generation-inference/issues/1028
}

llm_model = deploy_model(
    endpoint_name=llm_endpoint_name,
    instance_type="ml.g5.xlarge",
    env=llm_env,
    image_uri=llm_image_uri,
    wait=True,
)

Once the endpoint is deployed we can invoke it using the boto3 SDK. We will use the [Llama2 recommended prompt format](https://huggingface.co/blog/llama2#how-to-prompt-llama-2) along with a user instruction to create the final prompt which we will pass as a payload to the model. 

In [ ]:
# invoke llm
# first invocation is going to be slower. Subsequent ones should be faster
system_message = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""

prompt = "Why is New York City sometimes referred to as the Big Apple?"
prompt_template = f"""[INST] <<SYS>>{system_message}<</SYS>>\n{prompt}[/INST]"""

body = {
    "inputs": prompt_template,
    "parameters": {
        "max_new_tokens": 250,
        "temperature": 0.8,
        "return_full_text": False, # if True this will return our original prompt along with the generated text
    },
}
resp = smr.invoke_endpoint(
    EndpointName=llm_endpoint_name,
    Body=json.dumps(body),
    ContentType="application/json",
)
output = json.loads(resp["Body"].read().decode("utf-8"))
print(output[0]["generated_text"])

Run the code below to delete the endpoint and avoid any additional charges.

In [ ]:
# Clean up
sm.delete_endpoint(EndpointName=llm_endpoint_name)
sm.delete_endpoint_config(EndpointConfigName=llm_endpoint_name)

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.


![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-1/inference|generativeai|llm-workshop|deploy-gptq-quant-tgi|deploy_gptq_quant_tgi.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-2/inference|generativeai|llm-workshop|deploy-gptq-quant-tgi|deploy_gptq_quant_tgi.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-1/inference|generativeai|llm-workshop|deploy-gptq-quant-tgi|deploy_gptq_quant_tgi.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ca-central-1/inference|generativeai|llm-workshop|deploy-gptq-quant-tgi|deploy_gptq_quant_tgi.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/sa-east-1/inference|generativeai|llm-workshop|deploy-gptq-quant-tgi|deploy_gptq_quant_tgi.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-1/inference|generativeai|llm-workshop|deploy-gptq-quant-tgi|deploy_gptq_quant_tgi.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-2/inference|generativeai|llm-workshop|deploy-gptq-quant-tgi|deploy_gptq_quant_tgi.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-3/inference|generativeai|llm-workshop|deploy-gptq-quant-tgi|deploy_gptq_quant_tgi.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-central-1/inference|generativeai|llm-workshop|deploy-gptq-quant-tgi|deploy_gptq_quant_tgi.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-north-1/inference|generativeai|llm-workshop|deploy-gptq-quant-tgi|deploy_gptq_quant_tgi.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-1/inference|generativeai|llm-workshop|deploy-gptq-quant-tgi|deploy_gptq_quant_tgi.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-2/inference|generativeai|llm-workshop|deploy-gptq-quant-tgi|deploy_gptq_quant_tgi.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-1/inference|generativeai|llm-workshop|deploy-gptq-quant-tgi|deploy_gptq_quant_tgi.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-2/inference|generativeai|llm-workshop|deploy-gptq-quant-tgi|deploy_gptq_quant_tgi.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-south-1/inference|generativeai|llm-workshop|deploy-gptq-quant-tgi|deploy_gptq_quant_tgi.ipynb)
